<a href="https://colab.research.google.com/github/lizardnote/ML_practice/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_ch3_2_LR_KNN_DT_RF_SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.회귀분석
- KNN
- DT

In [14]:
#data load
import pandas as pd
import numpy as np

train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/st_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/st_test.csv')

# train_x = train.drop(['grade'], axis = 1)
# train_y = train['grade']
test_x = test.drop(['grade'], axis = 1)
test_y = test['grade']

In [15]:
if 'grade' in train.columns:
    train_x = train.drop(['grade'], axis=1)
    train_y = train['grade']
else:
    print("❌ 'grade' 컬럼이 존재하지 않습니다.")

In [20]:
#data preprocessing

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV

#컬럼 나눠서 선택
num_col = train_x.select_dtypes('number').columns
cat_col = train_x.select_dtypes('object').columns

#범주형 파이프라인
cat_preprocess = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore', sparse_output = False)
)
#수치형 파이프라인
num_preprocess = make_pipeline(
    SimpleImputer(strategy = "mean"),
    StandardScaler()
)
#한번에 합쳐서 처리
preprocess = ColumnTransformer([
    ('num', num_preprocess, num_col),
    ('cat', cat_preprocess, cat_col)
])

## K-Nearest Neighbors (KNN)

In [21]:
from sklearn.neighbors import KNeighborsRegressor

full_pipe = Pipeline(
    [ ('preprocess', preprocess),
      ('regressor', KNeighborsRegressor())
    ]
)

In [22]:
if 'grade' in train.columns:
    train_x = train.drop(['grade'], axis=1)
    train_y = train['grade']
else:
    print("❌ 'grade' 컬럼이 존재하지 않습니다.")

In [23]:
knn_param = {'regressor__n_neighbors': np.arange(5,10,1)}

knn_search = GridSearchCV(estimator = full_pipe,
                          param_grid = knn_param,
                          cv = 3,
                          scoring = 'neg_mean_squared_error',
                          error_score='raise')

knn_search.fit(train_x, train_y)

GridSearchCV(cv=3, error_score='raise',
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         Index(['school', 'sex', 'paid'], dtype='object'))])),
                                       ('regressor', KNeighborsRegressor())]),
             param_grid={'regressor__n_neighbors': array([5, 6, 7, 8, 9])},
             scoring='neg_mean_squared_error')

In [24]:
pd.DataFrame(knn_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013919,0.004547,0.008271,0.000501,5,{'regressor__n_neighbors': 5},-9.297209,-8.924235,-10.583059,-9.601501,0.710572,4
1,0.010841,0.001375,0.008307,0.000756,6,{'regressor__n_neighbors': 6},-9.269703,-8.851634,-10.933987,-9.685108,0.899433,5
2,0.010060,0.000136,0.007566,0.000211,7,{'regressor__n_neighbors': 7},-9.094922,-8.545738,-10.694118,-9.444926,0.911322,3
3,0.010149,0.000661,0.007680,0.000084,8,{'regressor__n_neighbors': 8},-9.122093,-8.554963,-10.609926,-9.428994,0.866549,2
4,0.010839,0.001628,0.008225,0.000834,9,{'regressor__n_neighbors': 9},-9.002010,-8.557008,-10.621932,-9.393650,0.887323,1


In [25]:
print('best parameter :', knn_search.best_params_)
print('best score :', knn_search.best_score_)

best parameter : {'regressor__n_neighbors': np.int64(9)}
best score : -9.39364982857915


In [27]:
#최종 평가
from sklearn.metrics import mean_squared_error

knn_pred = knn_search.predict(test_x)
print('test MSE : ', mean_squared_error(test_y, knn_pred))


test MSE :  9.736363636363635


## Decision Tree

In [33]:
from sklearn.tree import DecisionTreeRegressor
full_pipe = Pipeline(
    [
        ("preprocess", preprocess),
        ("regressor", DecisionTreeRegressor())
    ]
)

In [34]:
DecisionTreeRegressor().get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': None,
 'splitter': 'best'}

In [35]:
dt_param = {'regressor__ccp_alpha':np.arange(0.01, 0.3, 0.05)}
dt_search = GridSearchCV(estimator = full_pipe, param_grid = dt_param,
                         cv = 5, scoring = 'neg_mean_squared_error')
dt_search.fit(train_x, train_y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         Index(['school', 'sex', 'paid'], dtype='object'))])),
                                       ('regressor', DecisionTreeRegressor())]),
             param_grid={'regressor__ccp_alpha': array([0.01, 0.06, 0.11, 0.16, 0.21, 0.26])},
             scoring='neg_mean_squared_error')

In [36]:
print('best parameter :', dt_search.best_params_)
print('best score :', dt_search.best_score_)

best parameter : {'regressor__ccp_alpha': np.float64(0.26)}
best score : -9.403541096157653


In [38]:
from sklearn.metrics import mean_squared_error
df_pred = dt_search.predict(test_x)
print('test MSE : ', mean_squared_error(test_y, df_pred))

test MSE :  10.23195890566565


# 앙상블 학습
- Bagging
- Random Forest
- Gradient Boosting

In [43]:
from sklearn.ensemble import BaggingRegressor

full_pipe = Pipeline(
    [
        ("preprocess", preprocess),
        ("regressor", BaggingRegressor())
    ]
)


In [44]:
BaggingRegressor().get_params()

{'bootstrap': True,
 'bootstrap_features': False,
 'estimator': None,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [45]:
bagging_param = {'regressor__n_estimators': np.arange(10,100,20),
                 'regressor__random_state' : [0]}
bagging_search = GridSearchCV( estimator = full_pipe,
                               param_grid = bagging_param,
                               cv = 5,
                               scoring = 'neg_mean_squared_error')

bagging_search.fit(train_x, train_y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         Index(['school', 'sex', 'paid'], dtype='object'))])),
                                       ('regressor', BaggingRegressor())]),
             param_grid={'regressor__n_estimators': array([10, 30, 50, 70, 90]),
                         'regressor__random_state': [0]},
             scoring='neg_mean_squared_error')

In [46]:
print('best parameter:', bagging_search.best_params_)
print('MSE:', -bagging_search.best_score_)


best parameter: {'regressor__n_estimators': np.int64(30), 'regressor__random_state': 0}
MSE: 9.581004443482522


In [47]:
from sklearn.metrics import mean_squared_error

bag_pred = bagging_search.predict(test_x)
print('testMSE:', mean_squared_error(test_y, bag_pred))

testMSE: 9.626060816498317


In [49]:
#Random Forest

from sklearn.ensemble import RandomForestRegressor

full_pipe = Pipeline([
                      ("preprocess" ,preprocess),
                      ("regressor", RandomForestRegressor())
,])

In [50]:
RandomForestRegressor().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [55]:
rf_param = {'regressor__n_estimators' : np.arange(100,500,100),
            'regressor__max_features' : ['sqrt'],'regressor__random_state' : [0]}

rf_search = GridSearchCV(estimator = full_pipe,
                         param_grid = rf_param,
                         cv = 5,
                         scoring = 'neg_mean_squared_error')
rf_search.fit(train_x, train_y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         Index(['school', 'sex', 'paid'], dtype='object'))])),
                                       ('regressor', RandomForestRegressor())]),
             param_grid={'regressor__max_features': ['sqrt'],
                         'regressor__n_estimators': array([100, 200, 300, 400]),
                         'regressor__random_state': [0]},
             scoring='neg_mean_squared_error')

In [56]:
print('best parameter:', rf_search.best_params_)
print('MSE:', -rf_search.best_score_)

best parameter: {'regressor__max_features': 'sqrt', 'regressor__n_estimators': np.int64(400), 'regressor__random_state': 0}
MSE: 9.418447711599324


In [58]:
from sklearn.metrics import mean_squared_error

rf_pred = rf_search.predict(test_x)
print('MSE:', mean_squared_error(test_y, rf_pred))

MSE: 10.007126272089838


In [59]:
#gradient boosting : 약학습기를 생성하여 학습하면서 잔차를 업데이트함

from sklearn.ensemble import GradientBoostingRegressor

full_pipe = Pipeline([
                      ("preprocess" ,preprocess),
                      ("regressor", GradientBoostingRegressor())

                      ])


In [60]:
GradientBoostingRegressor().get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [65]:
GradientBoosting_param = {'regressor__learning_rate' : np.arange(0.1,0.3,0.05),
                          'regressor__random_state' : [0]}
GradientBoosting_search = GridSearchCV(estimator = full_pipe,
                                       param_grid = GradientBoosting_param,
                                       cv = 5,
                                       scoring = 'neg_mean_squared_error')
GradientBoosting_search.fit(train_x, train_y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         Index(['school', 'sex', 'paid'], dtype='object'))])),
                                       ('regressor',
                                        GradientBoostingRegressor())]),
             param_grid={'regressor__learning_rate': array([0.1 , 0.15, 0.2 , 0.25]),
                         'regressor__random_state': [0]},
             scoring='neg_mean_squared_error')

In [66]:
print('best parameter:', GradientBoosting_search.best_params_)
print('MSE:', -GradientBoosting_search.best_score_)

best parameter: {'regressor__learning_rate': np.float64(0.1), 'regressor__random_state': 0}
MSE: 10.788240022704802


In [67]:
from sklearn.metrics import mean_squared_error

GB_pred = GradientBoosting_search.predict(test_x)
print('MSE:', mean_squared_error(test_y, GB_pred))

MSE: 10.547041848465328


# SVR : Support Vector Regression

In [75]:
from sklearn.svm import SVR

full_pipe = Pipeline([
       ('preprocess', preprocess), #앞단은 변환기 (transformer)
        ('regressor', SVR())          #마지막단은 예측기(estimator)
])

- 예측기 (fit/predict 둘 다 가짐)
- 변환기 (transform만 가짐)
파이프라인 구조는 모듈화된 전처리 + 모델학습 흐름을 위한 고정된 규칙이므로 순서 지켜야함

In [76]:
SVR().get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [77]:
svr_param = {'regressor__C': np.arange(1,100,20)}
svr_search = GridSearchCV( estimator = full_pipe,
                           param_grid = svr_param,
                           cv = 5,
                           scoring = 'neg_mean_squared_error')

svr_search.fit(train_x, train_y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         Index(['school', 'sex', 'paid'], dtype='object'))])),
                                       ('regressor', SVR())]),
             param_grid={'regressor__C': array([ 1, 21, 41, 61, 81])},
             scoring='neg_mean_squared_error')

In [78]:
print('bsst parameter : ', svr_search.best_params_)
print('MSE : ', -svr_search.best_score_)

bsst parameter :  {'regressor__C': np.int64(1)}
MSE :  8.905507590431395


In [80]:
from sklearn.metrics import mean_squared_error

svr_pred = svr_search.predict(test_x)
print('MSE : ', mean_squared_error(test_y, svr_pred))

MSE :  10.141966042523617


# 작업2유형 모범 답안 예시

In [82]:
#data load

import pandas as pd
import numpy as np
train = pd.read_csv('http://raw.githubusercontent.com/YoungjinBD/data/main/st_train.csv')
test = pd.read_csv('http://raw.githubusercontent.com/YoungjinBD/data/main/st_test.csv')

In [83]:
#chechk data info : data type, Nan

print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   school    256 non-null    object 
 1   sex       256 non-null    object 
 2   paid      256 non-null    object 
 3   famrel    256 non-null    int64  
 4   freetime  256 non-null    int64  
 5   goout     252 non-null    float64
 6   Dalc      256 non-null    int64  
 7   Walc      256 non-null    int64  
 8   health    256 non-null    int64  
 9   absences  256 non-null    int64  
 10  grade     256 non-null    int64  
dtypes: float64(1), int64(7), object(3)
memory usage: 22.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   school    110 non-null    object 
 1   sex       110 non-null    object 
 2   paid      110 non-null    object 
 3   famrel    110 n

In [84]:
#data split
#target split
train_x = train.drop(['grade'], axis =1)
train_y = train['grade']
test_x = test.drop(columns = 'grade')
test_y = test['grade']

#validation set split from train set
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.2, random_state = 0)

In [91]:
#feature split for categorical / number
cat_col = train_x.select_dtypes('object').columns.to_list()
num_col = train_x.select_dtypes('number').columns.to_list()

# one hot encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

#컬럼별 인코딩을 위한 메소드 선언(원핫인코딩, 평균대치법)
onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')
imputer = SimpleImputer(strategy='mean')

#train, valid, test set 전처리
train_x_numeric_imputed = imputer.fit_transform(train_x[num_col])
train_valid_x_numeric_imputed = imputer.transform(val_x[num_col])
test_x_numeric_impputed = imputer.transform(test_x[num_col])

train_x_cat_encoded = onehotencoder.fit_transform(train_x[cat_col])
train_valid_x_cat_encoded = onehotencoder.transform(val_x[cat_col])
test_x_cat_encoded = onehotencoder.transform(test_x[cat_col])

#후 다시 결합
train_x_preprocessed = np.concatenate([train_x_numeric_imputed, train_x_cat_encoded], axis = 1)
valid_x_preprocessed = np.concatenate([train_valid_x_numeric_imputed, train_valid_x_cat_encoded], axis = 1)
test_x_preprocessed = np.concatenate([test_x_numeric_impputed, test_x_cat_encoded], axis = 1)

In [92]:
#model
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
rf.fit(train_x_preprocessed, train_y)
RandomForestRegressor(random_state = 42)


RandomForestRegressor(random_state=42)

In [99]:
from sklearn.metrics import mean_squared_error
pred_val = rf.predict(valid_x_preprocessed)
mse = mean_squared_error(val_y, pred_val)
rmse = np.sqrt(mse)
print('RMSE:', rmse)

RMSE: 3.1911433571715966


In [101]:
test_pred = rf.predict(test_x_preprocessed)
test_pred = pd.DataFrame(test_pred, columns = ['pred'])
test_pred.to_csv('result.csv', index = False)

In [105]:
train_x_full = np.concatenate([train_x_preprocessed, valid_x_preprocessed])
train_y_full = np.concatenate([train_y, val_y], axis = 0)

In [110]:
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': [10,20,30],
              'min_samples_split' : [2,5,10]}

rf = RandomForestRegressor(random_state=42)
rf_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, scoring = 'neg_mean_squared_error')
rf_search.fit(train_x_full, train_y_full)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_split': [2, 5, 10]},
             scoring='neg_mean_squared_error')

In [113]:
print('RMSE :', -rf_search.best_score_)

RMSE : 8.59111281873375


In [115]:
test_pred2 = rf_search.predict(test_x_preprocessed)
test_pred2 = pd.DataFrame(test_pred2, columns = ['pred'])
test_pred2.to_csv('result2.csv', index = False)